In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
from keras.utils import HDF5Matrix
from PIL import Image

import tensorflow as tf
import keras
from keras.callbacks import Callback
from keras import backend as K
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array

from keras import Model
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import SGD, Adam
from keras.metrics import categorical_crossentropy

from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input


import os
import time
from IPython import display

#print(os.listdir("../input"))

# directory to save the best model and last model
file_dir = '../working/Model'
os.mkdir(file_dir)

Using TensorFlow backend.


## Import Dataset

In [2]:
# Import dataset
x_train = HDF5Matrix('../input/camelyonpatch_level_2_split_train_x.h5-002/camelyonpatch_level_2_split_train_x.h5-002', 'x')
mask_train = HDF5Matrix('../input/camelyonpatch_level_2_split_train_mask.h5/camelyonpatch_level_2_split_train_mask.h5', 'mask')
y_train = HDF5Matrix('../input/camelyonpatch_level_2_split_train_y.h5/camelyonpatch_level_2_split_train_y.h5', 'y')               
x_val = HDF5Matrix('../input/camelyonpatch_level_2_split_valid_x.h5/camelyonpatch_level_2_split_valid_x.h5', 'x')
y_val = HDF5Matrix('../input/camelyonpatch_level_2_split_valid_y.h5/camelyonpatch_level_2_split_valid_y.h5', 'y')
x_test = HDF5Matrix('../input/camelyonpatch_level_2_split_test_x.h5/camelyonpatch_level_2_split_test_x.h5', 'x')


In [3]:
# Check dimensions
print("Training set")
print("img:",x_train.shape)
print("labels:",y_train.shape)
print('-'*40)
print("Validation set")
print("img:",x_val.shape)
print("labels:",y_val.shape)
print('-'*40)
print("Test set")
print("img:",x_test.shape)

Training set
img: (262144, 96, 96, 3)
labels: (262144, 1, 1, 1)
----------------------------------------
Validation set
img: (32768, 96, 96, 3)
labels: (32768, 1, 1, 1)
----------------------------------------
Test set
img: (32768, 96, 96, 3)


## Data Generator

In [4]:
# indexes to feed the generator
train_id = np.arange(262144)
val_id = np.arange(32768)

# creat a useful dictionary structures
partition = {}
partition['train'] = train_id
partition['validation'] = val_id

train_labels = {}
val_labels = {}

for i in range(len(train_id)):
    
    train_labels[str(i)] = y_train[i].flatten()[0] 
    
for i in range(len(val_id)):
    
    val_labels[str(i)] = y_val[i].flatten()[0]

In [5]:
#Data Generator to efficiently load and preprocess data for training the classifier

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, data, labels, batch_size=32, dim=(224, 224), n_channels=3,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.data = data
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, self.n_classes))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            
            # preprocessing
            img_arr = self.data[ID]
            img = array_to_img(img_arr)
            img = img.resize((224,224), Image.ANTIALIAS)
            img.load()
            
            X[i] = preprocess_input(np.asarray(img, dtype=np.uint8))
            
            
            

            # Store target label(one-hot-encoding)
            y[i] = to_categorical(self.labels[str(ID)], num_classes=self.n_classes)

        return X, y

## Callbacks

In [6]:
#Callback: TimerCallback()
# this callback make sure to interrupt the training if a certain time limit is reached, saving the weigths of the last model to train again

class TimerCallback(Callback):
    
    def __init__(self, maxExecutionTime, byBatch = False, on_interrupt=None):
        
# Arguments:
#     maxExecutionTime (number): Time in minutes. The model will keep training 
#                                until shortly before this limit
#                                (If you need safety, provide a time with a certain tolerance)

#     byBatch (boolean)     : If True, will try to interrupt training at the end of each batch
#                             If False, will try to interrupt the model at the end of each epoch    
#                            (use `byBatch = True` only if each epoch is going to take hours)          

#     on_interrupt (method)          : called when training is interrupted
#         signature: func(model,elapsedTime), where...
#               model: the model being trained
#               elapsedTime: the time passed since the beginning until interruption   

        
        self.maxExecutionTime = maxExecutionTime * 60
        self.on_interrupt = on_interrupt
        
        #the same handler is used for checking each batch or each epoch
        if byBatch == True:
            #on_batch_end is called by keras every time a batch finishes
            self.on_batch_end = self.on_end_handler
        else:
            #on_epoch_end is called by keras every time an epoch finishes
            self.on_epoch_end = self.on_end_handler
    
    
    #Keras will call this when training begins
    def on_train_begin(self, logs):
        self.startTime = time.time()
        self.longestTime = 0            #time taken by the longest epoch or batch
        self.lastTime = self.startTime  #time when the last trained epoch or batch was finished
    
    
    #this is our custom handler that will be used in place of the keras methods:
        #`on_batch_end(batch,logs)` or `on_epoch_end(epoch,logs)`
    def on_end_handler(self, index, logs):
        
        currentTime      = time.time()                           
        self.elapsedTime = currentTime - self.startTime    #total time taken until now
        thisTime         = currentTime - self.lastTime     #time taken for the current epoch
                                                               #or batch to finish
        
        self.lastTime = currentTime
        
        #verifications will be made based on the longest epoch or batch
        if thisTime > self.longestTime:
            self.longestTime = thisTime
        
        
        #if the (assumed) time taken by the next epoch or batch is greater than the
            #remaining time, stop training
        remainingTime = self.maxExecutionTime - self.elapsedTime
        if remainingTime < self.longestTime:
            
            self.model.stop_training = True  #this tells Keras to not continue training
            print("\n\nTimerCallback: Finishing model training before it takes too much time. (Elapsed time: " + str(self.elapsedTime/60.) + " minutes )\n\n")
            
            #if we have passed the `on_interrupt` callback, call it here
            if self.on_interrupt is not None:
                self.on_interrupt(self.model, self.elapsedTime)

In [7]:
#Callback: PlotCurves
# plot losses and accuracy after each epoch and save the weights of the best model
class PlotCurves(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.epoch = 0
        self.best_epoch = 0
        self.x = []
        self.losses = []
        self.acc = []
        self.val_losses = []
        self.val_acc = []
        self.best_val_acc = 0
        self.fig = plt.figure(figsize=(10, 5))
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.fig = plt.figure(figsize=(10, 5))
        self.logs.append(logs)
        self.x.append(self.epoch)
        self.losses.append(logs.get('loss'))
        self.acc.append(logs.get('acc'))
        self.val_losses.append(logs.get('val_loss'))
        self.val_acc.append(logs.get('val_acc'))
        self.epoch += 1
        
        # (Possibly) update best validation accuracy and save the network
        if self.val_acc[-1] > self.best_val_acc:
            self.best_val_acc = self.val_acc[-1]
            self.best_epoch = self.epoch
            self.model.save_weights(os.path.join(file_dir, 'best_model.h5'))
        
        display.clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.plot(self.x, self.acc, label="acc")
        plt.plot(self.x, self.val_acc, label="val_acc")
        plt.legend()
        plt.title('Best validation accuracy = {:.2f}% on epoch {} of {}'.format(100. * self.best_val_acc, self.best_epoch, self.epoch))
        plt.show();

## Model

In [8]:
# create a copy of a ResNet50 model

res50 = ResNet50(weights='imagenet', include_top=True)

Instructions for updating:
Colocations handled automatically by placer.
102858752/102853048 [==============================] - 4s 0us/step


In [9]:
res50.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [10]:
# remove last layer
res50.layers.pop()
res50_notop = Model(res50.input, res50.layers[-1].output)
#res50_notop.summary()

In [11]:
# function to replace last layer with a two-classes softmax layer and set how many layers to train

def res50_model(model=res50_notop, num_classes=2):

    x=Dense(num_classes, activation='softmax')(model.output)

    model=Model(model.input,x)

    #To set the first 161 layers to non-trainable (weights will not be updated)

    for layer in model.layers[:164]: # up to add_15 layer
        
        layer.trainable = False


    return model

In [12]:
# Compile the model

# parameters
num_classes = 2
lr = 0.01
loss = 'categorical_crossentropy'
metric = 'acc'

model = res50_model(res50_notop, num_classes)
model.compile(Adam(lr=lr, beta_1=0.9, beta_2=0.999,), loss=loss, metrics=[metric])
#model.summary()

## Train the model

In [14]:
# Define batch size.
batch_size = 64
n_epoch = 1000 
time_limit = 500 # time limit in minute (500 -> 8 hrs and 20 min)

# Parameters for generators
params = {'dim': (224, 224),
          'batch_size': batch_size,
          'n_classes': 2,
          'shuffle': True}

# Generators
training_generator = DataGenerator(partition['train'], x_train, train_labels, **params)
validation_generator = DataGenerator(partition['validation'], x_val, val_labels, **params)

#a function to save last_model compatible with the on_interrupt handler of TimerCallback
def saveWeights(model, elapsed):
    model.save_weights(os.path.join(file_dir, 'last_model.h5'))


# Callbacks
callbacks = [PlotCurves(), TimerCallback(1, on_interrupt=saveWeights)]


In [ ]:
# Training
model.fit_generator(training_generator, validation_data=validation_generator, epochs=n_epoch, callbacks=callbacks)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1000
 593/4096 [===>..........................] - ETA: 18:43 - loss: 0.4052 - acc: 0.8609